In [1]:
from sklearn.datasets import make_classification

In [2]:
base_dataset = make_classification(
    n_samples=10_000, 
    n_features=30, 
    n_informative=10,
    n_clusters_per_class=2,
    n_classes=4)

x, y = base_dataset

In [3]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

x_standardized = ss.fit_transform(x)

In [4]:
base_classes = []

for i in range(4):
    base_classes.append(x_standardized[y == i])

In [5]:
import numpy as np

num_points = 5_000
class1_dist = [.5, .5, 0, 0]
class2_dist = [0, .2, .6, .2]

def make_var_len_feature_point(dist):
    feature_sets = []
    num_features = np.random.randint(3, 11)
    for _ in range(num_features):
        # choose which distribution the credit card comes from
        base_class = np.random.choice([0, 1, 2, 3], 1, p=dist)
        base_class_points = base_classes[base_class[0]]
        feature_set_idx = np.random.choice(base_class_points.shape[0], 1)
        feature_sets.append(base_class_points[feature_set_idx])
        
    for _ in range(10 - num_features):
        feature_sets.append(np.zeros((1, 30)))

    return np.concatenate(feature_sets)[np.newaxis, :, :]


class1_points = []
for _ in range(num_points):
    class1_points.append(
        make_var_len_feature_point(class1_dist))
class1_points = np.concatenate(class1_points)
    
class2_points = []
for _ in range(num_points):
    class2_points.append(
        make_var_len_feature_point(class2_dist))
class2_points = np.concatenate(class2_points)

In [6]:
class2_points.shape

(5000, 10, 30)

In [7]:
def bootstrap_sample_generator(batch_size):
    while True:
        batch_idx = np.random.choice(
            class1_points.shape[0], batch_size // 2)
        batch_x = np.concatenate([
            class1_points[batch_idx],
            class2_points[batch_idx],
        ])
        batch_y = np.concatenate([
            np.zeros(batch_size // 2),
            np.ones(batch_size // 2),
        ])
        yield ({'numeric_inputs': batch_x}, 
               {'output': batch_y})

In [8]:
import tensorflow as tf

p = .1

In [9]:
inputs = tf.keras.layers.Input((10, 30), name='numeric_inputs')

In [12]:
x = tf.keras.layers.Dropout(p)(inputs)

x = tf.keras.layers.Conv1D(10, 1)(x)
x = tf.keras.layers.Activation('relu')(x)

global_ave = tf.keras.layers.GlobalAveragePooling1D()(x)
global_max = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.keras.layers.Concatenate()([global_ave, global_max])

x = tf.keras.layers.BatchNormalization()(x)

In [13]:
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(100, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(20, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
x = tf.keras.layers.Dense(10, activation='relu')(x)

x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(p)(x)
out = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(x)

In [14]:
model = tf.keras.models.Model(inputs=inputs, outputs=out)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 numeric_inputs (InputLayer)    [(None, 10, 30)]     0           []                               
                                                                                                  
 dropout_1 (Dropout)            (None, 10, 30)       0           ['numeric_inputs[0][0]']         
                                                                                                  
 conv1d_1 (Conv1D)              (None, 10, 10)       310         ['dropout_1[0][0]']              
                                                                                                  
 activation (Activation)        (None, 10, 10)       0           ['conv1d_1[0][0]']               
                                                                                              

In [16]:
batch_size = 32

model.fit_generator(
    bootstrap_sample_generator(batch_size),
    steps_per_epoch=10_000 // batch_size,
    epochs=5,
    max_queue_size=10,
)

C:\Users\serge\AppData\Local\Temp\ipykernel_29812\3518603723.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5
312/312 [==============================] - 3s 2ms/step - loss: 0.5634 - accuracy: 0.7007
Epoch 2/5
312/312 [==============================] - 1s 2ms/step - loss: 0.2958 - accuracy: 0.8777
Epoch 3/5
312/312 [==============================] - 1s 2ms/step - loss: 0.2332 - accuracy: 0.9051
Epoch 4/5
312/312 [==============================] - 1s 2ms/step - loss: 0.2102 - accuracy: 0.9163
Epoch 5/5
312/312 [==============================] - 1s 2ms/step - loss: 0.1914 - accuracy: 0.9231
